<a href="https://colab.research.google.com/github/RomanKunal/Deep-Learning/blob/main/Sentimental_Analysis_of_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kaggle

In [3]:
import os
import json

In [4]:
from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
#!/bin/bash
!kaggle datasets download lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 89% 23.0M/25.7M [00:00<00:00, 92.3MB/s]
100% 25.7M/25.7M [00:00<00:00, 85.7MB/s]


In [6]:
#unzip the file
path='/content/imdb-dataset-of-50k-movie-reviews.zip'
with ZipFile(path,'r') as zip:
  zip.extractall()
  print('done')

done


In [7]:
data=pd.read_csv('IMDB Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [8]:
data.shape

(50000, 2)

In [9]:
data.isnull().sum()

,0
review,0
sentiment,0


In [10]:
data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [11]:
mapping_dict = {'positive': 1, 'negative': 0}
data['sentiment'] = data['sentiment'].map(mapping_dict)


In [12]:
data['sentiment'].value_counts()

,count
sentiment,
1,25000
0,25000


In [13]:
#split the data
train_data,test_data=train_test_split(data,test_size=0.2,random_state=42)

In [14]:
train_data.shape

(40000, 2)

In [17]:
#tokenize the text data
tokenizer=Tokenizer(num_words=5000)

#fit the tokenizer
tokenizer.fit_on_texts(train_data['review'])

#convert text to sequences
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)







In [18]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [19]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [20]:
Y_train=train_data['sentiment']
Y_test=test_data['sentiment']

In [21]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


In [32]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))  # input_length defined
model.add(LSTM(units=128, dropout=0.2))
model.add(Dense(units=1, activation='sigmoid'))

# Build the model
model.build(input_shape=(None, 200))  # Batch size unspecified


In [33]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ (None, 200, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [34]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [35]:
model.fit(X_train, Y_train, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.7039 - loss: 0.5503 - val_accuracy: 0.8615 - val_loss: 0.3317
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.8798 - loss: 0.2984 - val_accuracy: 0.8737 - val_loss: 0.3089
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.9103 - loss: 0.2313 - val_accuracy: 0.8699 - val_loss: 0.3089
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.9281 - loss: 0.1934 - val_accuracy: 0.8823 - val_loss: 0.3151
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.9383 - loss: 0.1594 - val_accuracy: 0.8704 - val_loss: 0.3342
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9518 - loss: 0.1286 - val_accuracy: 0.8621 - val_loss: 0.3613
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.9553 - loss: 0.1198 - val_accuracy: 0.8759 - val_loss: 0.3700
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.9673 - loss: 0.0962 - val

In [36]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8733 - loss: 0.4490
Test Loss: 0.4546281397342682
Test Accuracy: 0.876800000667572


In [37]:
#building the predictive system
def pred(review):
  seq=tokenizer.texts_to_sequences([review])
  padded=pad_sequences(seq,maxlen=200)
  pred=model.predict(padded)
  if pred>0.5:
    print('positive')
  else:
    print('negative')

In [38]:
#example
pred('this movie was amazing but kinds slow paced')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
positive
